In [1]:
import pandas as pd

url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"

sharks_df = pd.read_excel(url)

#Dataframed based on the relevant columns
sharks_df = sharks_df.drop(columns=["Sex", "Age", "Name", "Injury", "Source", "pdf",'href',"href formula",'Case Number','Case Number.1','original order','Unnamed: 21','Unnamed: 22','Time'])
sharks_df.columns


Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Fatal Y/N', 'Species '],
      dtype='object')

In [2]:
#Make columns lower case for readibility.
sharks_df.rename(columns={ col: col.rstrip().replace(" ", "_").lower() for col in sharks_df.columns}, inplace=True)
sharks_df

column_list = ['date','type','country','state','location','activity','fatal_y/n','species']
column_list
#Remove null values
def remove_null(column_list, value):
    """
    To remove nulls from a column, add the column name in the first parameter and
    then the value to be replaced with.
    """
    for col in column_list:
      sharks_df[col] = sharks_df[col].fillna(value)
      print(f"Null values in column '{col}' have been replaced with value: {value}.")
    return sharks_df[col]

remove_null(column_list,'N/A') #code works


# #Amount of null values
sharks_df.isna().sum() #Output: no null values

Null values in column 'date' have been replaced with value: N/A.
Null values in column 'type' have been replaced with value: N/A.
Null values in column 'country' have been replaced with value: N/A.
Null values in column 'state' have been replaced with value: N/A.
Null values in column 'location' have been replaced with value: N/A.
Null values in column 'activity' have been replaced with value: N/A.
Null values in column 'fatal_y/n' have been replaced with value: N/A.
Null values in column 'species' have been replaced with value: N/A.


date         0
year         2
type         0
country      0
state        0
location     0
activity     0
fatal_y/n    0
species      0
dtype: int64

In [3]:
#Check for special characters in a column
def check_char(column_data):
  """
  Function to check for special characters using regen. Please name one column to check for special characters
  """
  result = column_data.str.contains(r'[^a-zA-Z\s]', regex=True, na=False).sum()
  table = column_data.str.contains(r'[^a-zA-Z\s]', regex=True, na=False)
  print(f"The number of values with special characters in this column is: {result}")
  return column_data[table].head(50)

def clean_fatal_rest(x):
    if x not in ["N", "Y"]:
        return "N/A"
    else:
        return x

#Removing invalid values for fatal_y/n column and setting it only for 'N', 'Y', or 'N/A'
sharks_df['fatal_y/n'] = sharks_df['fatal_y/n'].str.strip().str.upper().str.replace(r'[^a-zA-Z]', '', regex=True)
sharks_df['fatal_y/n'] = sharks_df['fatal_y/n'].apply(clean_fatal_rest)
sharks_df['fatal_y/n'].value_counts()

# check_char(sharks_df['fatal_y/n'])
sharks_df['fatal_y/n'].value_counts()

fatal_y/n
N      4879
Y      1469
N/A     642
Name: count, dtype: int64

In [4]:
#Date column conversion
column_list = ['date']
#Converting 'date' column to datetime and converting invalid dates to NaT
sharks_df['date'] = pd.to_datetime(sharks_df['date'], errors='coerce')
#Setting a dataframe for invalid dates equal to N/A
invalid_dates = sharks_df[sharks_df['date'].isna()]
#setting a variable and applying function to clean up NAs
cleaned_date_column = remove_null(column_list,'N/A')
#Appling it to date column
sharks_df['date'] = cleaned_date_column
#extracting the month in a new column
sharks_df['month'] = sharks_df['date'].apply(lambda x: x.month if isinstance(x, pd.Timestamp) else x)
#determining the season of the year based on month column
sharks_df['season'] = sharks_df['month'].apply(
    lambda x: 'Spring' if isinstance(x, int) and 3 <= x <= 5 else
              ('Summer' if isinstance(x, int) and 6 <= x <= 8 else
               ('Fall' if isinstance(x, int) and 9 <= x <= 11 else
                'Winter' if isinstance(x, int) else 'N/A')))
#formatting the date column to YYYY-MM-DD
sharks_df['date'] = sharks_df['date'].apply(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x)
sharks_df['season'].value_counts()

display(sharks_df)




Null values in column 'date' have been replaced with value: N/A.


,date,year,type,country,state,location,activity,fatal_y/n,species,month,season
0,2025-01-11,2025.0,Provoked,USA,Hawaii,Off Haleiwa Boat Harbour Oahu,Diving,N,Not stated,1,Winter
1,2025-01-02,2025.0,Unprovoked,New Caledonia,Grande Terre,Islet of Kendek near Koumac,Spearfishing,Y,Reportedly Tiger or Bull shark,1,Winter
2,2025-01-02,2025.0,Unprovoked,Australia,South Australia,Granites Beach near Westall Streaky Bay,Surfing,Y,Great White Shart estimated 4.8m 16 ft,1,Winter
3,2024-12-29,2024.0,Unprovoked,Egypt,North of Marsa Alam,Red Sea,SCUBA Diving,N,Reportedly a Tiger Shark,12,Winter
4,2024-12-29,2024.0,Unprovoked,Egypt,North of Marsa Alam,Red Sea,SCUBA Diving,Y,Reportedly a Tiger Shark,12,Winter
...,...,...,...,...,...,...,...,...,...,...,...
6985,N/A,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,Y,N/A,N/A,N/A
6986,N/A,0.0,Unprovoked,AUSTRALIA,Western Australia,N/A,Pearl diving,Y,N/A,N/A,N/A
6987,N/A,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Y,N/A,N/A,N/A
6988,N/A,0.0,Unprovoked,PANAMA,N/A,"Panama Bay 8ºN, 79ºW",N/A,Y,N/A,N/A,N/A


In [5]:
#Conclusion: USA seems to be a safer place to offer tours compared to Australia. However, we could offer "caged" tours in Australia. Create a pivot to support the hypothesis.
group = sharks_df.groupby('fatal_y/n')
non_fatal_df = group.get_group('N')
fatal_df = group.get_group('Y')
sorted_df = non_fatal_df.groupby(['location','state','country']).size().reset_index(name='count')
sorted_df = sorted_df.sort_values(by='count', ascending=False)
sorted_df

,location,state,country,count
2065,"New Smyrna Beach, Volusia County",Florida,USA,188
739,"Daytona Beach, Volusia County",Florida,USA,31
2537,"Ponce Inlet, Volusia County",Florida,USA,28
1782,"Myrtle Beach, Horry County",South Carolina,USA,21
1663,"Melbourne Beach, Brevard County",Florida,USA,19
...,...,...,...,...
1257,Jimmy's Beach,New South Wales,AUSTRALIA,1
1258,"Jimmys Beach, Port Stephens",New South Wales,AUSTRALIA,1
1259,"Jogensfontein, Stilbaai",Western Cape Province,SOUTH AFRICA,1
1260,"John Pennekamp Coral Reef State Park, Monroe C...",Florida,USA,1


In [6]:
#Non fatal shark occurrences by season and country
group = sharks_df.groupby(['season','state'])
group.size().sort_values(ascending=False)
sorted_df = non_fatal_df.groupby(['season','country']).size().reset_index(name='count')
sorted_df = sorted_df.sort_values(by='count', ascending=False).head(30)
sorted_df

,season,country,count
311,Summer,USA,867
74,Fall,USA,575
230,Spring,USA,404
317,Winter,AUSTRALIA,390
2,Fall,AUSTRALIA,215
158,Spring,AUSTRALIA,214
401,Winter,USA,186
384,Winter,SOUTH AFRICA,174
234,Summer,AUSTRALIA,114
155,N/A,USA,114


In [7]:
#fatal shark occurrences by season and country
group = sharks_df.groupby(['season','state'])
group.size().sort_values(ascending=False)
sorted_df = fatal_df.groupby(['season','country']).size().reset_index(name='count')
sorted_df = sorted_df.sort_values(by='count', ascending=False).head(30)
sorted_df

,season,country,count
279,Winter,AUSTRALIA,120
271,Summer,USA,69
2,Fall,AUSTRALIA,59
153,Spring,AUSTRALIA,52
333,Winter,SOUTH AFRICA,46
61,Fall,USA,39
343,Winter,USA,39
65,N/A,AUSTRALIA,38
208,Summer,AUSTRALIA,33
146,N/A,USA,28


In [8]:
#Average shark attack per year in a given month
season_year_group = sharks_df.groupby(['month', 'year','country']).size().reset_index(name='count')
season_year_group.sort_values(by='year',ascending=False)
# avg_attacks_per_year = season_year_group.groupby('month')['count'].mean().reset_index(name='average_attacks')
# avg_attacks_per_year_sorted = avg_attacks_per_year.sort_values(by='average_attacks', ascending=False)
# avg_attacks_per_year_sorted

,month,year,country,count
2212,7,2026.0,USA,1
2211,7,2025.0,USA,1
529,1,2025.0,USA,1
528,1,2025.0,New Caledonia,1
527,1,2025.0,Australia,1
...,...,...,...,...
3608,N/A,0.0,BAHAMAS,3
3609,N/A,0.0,BAHREIN,1
3641,N/A,0.0,PAPUA NEW GUINEA,4
3639,N/A,0.0,PACIFIC OCEAN,1


In [10]:
#Location and County Clean-up

#Check for commas in the 'location' column before splitting
has_comma = sharks_df['location'].str.contains(',', na=False)

# Split only rows with commas
sharks_df = sharks_df.loc[has_comma, ['location', 'county']] = sharks_df.loc[has_comma, 'location'].str.split(',', n=1, expand=True)

# Clean up by stripping extra spaces
sharks_df['location'] = sharks_df['location'].str.strip()
sharks_df['county'] = sharks_df['county'].str.strip()

# Fill NaN values in 'county' for rows without commas
sharks_df['county'] = sharks_df['county'].fillna('Unknown')
sharks_df['county'] = sharks_df['county'].replace('Unknown', 'N/A')

sharks_df[['location','county']].value_counts().tail(50)

Series([], Name: count, dtype: int64)

In [ ]:
# Filter out 'In' or 'in' in locations
def filter_out_in(df, column='location', keyword='in'):
    """
    Filters out rows where the specified column contains the given keyword (case-insensitive).

    Args:
        df (pd.DataFrame): The DataFrame to filter.
        column (str): The column to search in.
        keyword (str): The keyword to filter out.

    Returns:
        pd.DataFrame: A filtered DataFrame excluding rows with the keyword.
    """
    # Exclude rows where the column contains the keyword
    filtered_df = df[~df[column].str.contains(keyword, case=False, na=False)]
    return filtered_df

# Filter out rows with 'In' or 'in' in the 'location' column
filtered_sharks_df = filter_out_in(sharks_df, column='location', keyword='in')

# Get the unique values in the 'location' column
unique_locations = filtered_sharks_df['location'].unique()

# Count the number of unique locations
number_of_unique_locations = len(unique_locations)

# Display the result
print(f"Number of unique locations: {number_of_unique_locations}")


In [ ]:
# Country clean-up

# Strip whitespace and standardize capitalization
sharks_df['country'] = sharks_df['country'].str.strip().str.title()

# Replace common errors or inconsistencies
country_replacements = {
    'Usa': 'USA',
    'U.S.A.': 'USA',
    'United States Of America': 'USA',
    'Aus': 'Australia',
    'Uk': 'United Kingdom',
    'South Africa ': 'South Africa'
}

#sharks_df['country'] = sharks_df['country'].replace(country_replacements)

# Handle missing values
sharks_df['country'] = sharks_df['country'].fillna('Unknown')

sharks_df['country'].nunique()